# Simple BSM Option Pricer
Summer 2022 <br>
Whitney Rueckl

**This notebook contains code for simple European option pricing.**

The **Black-Scholes-Merton (BSM)** option pricing framework assumes that the price process of the underlying asset follows a *geometric brownian motion (GBM)*. Therefore it assumes the following:

1. The stochastic price process is continuous.

1. The log returns are normally distributed.

2. The returns during any two disjoint periods are independent.

GBMs are one of the simplest stochastic processes that reasonably model asset price dynamics.

The price process of a geometric brownian motion is determined by the current risk-free rate $r$ and the annualized volatility of the underlying $\sigma$.  Prices that are separated by $\Delta t$ units of time are related by following equation:

$$S_{t} =  S_{t - \Delta t} \cdot \exp\bigg(\bigg(r - \frac{1}{2}\sigma^2\bigg)\Delta t + \sigma \sqrt{\Delta t} z_{t}\bigg)$$

where $z_{t}$ is a standard normal random variable.


In [1]:
import pandas as pd
import numpy as np
import math

# Supress scientific notation for large numbers
pd.options.display.float_format = '{:.2f}'.format

# OS - for a function to get the working directory
import os
import time

#matplotlib - functions for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

#render plotting automatically
%matplotlib inline

In [2]:
""" 
Stores common attributes of a stock option 
"""
class StockOption(object):
    def __init__(
        self, S0, K, r=0.05, T=1, N=2, div=0, sigma=0, is_put=False, n_paths=100):
        
        """
        Initialize the stock option base class (European option only).
        
        Parameters:
        
        S0: initial stock price
        K: strike price
        r: risk-free interest rate
        T: time to maturity
        N: number of time steps
        div: Dividend yield
        is_put: True for a put option,
                False for a call option            
        """

        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N)
        self.is_put = is_put
        #self.STs = [] # Declare the stock prices tree (array)
        self.n_paths = n_paths

        """ Optional parameters used by derived classes """
        self.div = div
        self.sigma = sigma
        self.is_call = not is_put # understand the not part and how thats different than


    @property
    def dt(self):
        """ single time step, in years """
        return self.T/float(self.N)

    @property
    def df(self):
        """ the discount factor """
        return math.exp(-(self.r-self.div)*self.dt)  

In [3]:
stock = StockOption(S0 = 50, K = 60, r=0.05, T=1, N=2,
        div=0, sigma=0, is_put=False, n_paths = 100)

In [4]:
stock.S0

50

In [5]:
stock.df

0.5

In [15]:
class BSMOptionPricer(StockOption):
    
    ''' 
    Inherits from the StockOption class.
    
    Methods:
    
    - simPaths: simulate price paths of the underlying using geometric brownian motion
    - calcPayoff: calculate the payoff of the option
    - calcPrice: calculates the options price    
    
    '''

    
    def simPaths(self):
        price_paths = np.zeros((self.N + 1, self.n_paths))
        price_paths[0] = self.S0

        for t in range(1, self.N + 1):
                z = np.random.standard_normal(self.n_paths)
                #print(z)
                price_paths[t] = price_paths[t - 1] * np.exp((self.r- 0.5 * self.sigma ** 2) * self.dt + self.sigma * np.sqrt(self.dt) * z) # memorize this line
                #price_paths[t] = np.round(price_paths[t], 2)
                #print(price_paths[t])

        return price_paths          

        
    def calcPayoff(self):
        
        #sim_paths = self.simPaths()
        self.sim_paths = self.simPaths()
        
        if self.is_put == False:        
            sim_payoff = np.maximum(0, (self.sim_paths[-1] - self.K))
        else:
            sim_payoff = np.maximum(0, (self.K - self.sim_paths[-1]))

        return sim_payoff 
    
    
    def calcPrice(self):
        
        payoff = self.calcPayoff()
        
        price = np.exp(-self.r * self.T) * np.sum(payoff) / self.n_paths
        
        return price

Instantiate the `BSMOPtionPricer`

In [16]:
optpricer = BSMOptionPricer(S0 = 50, K = 50, r=0.05, T=1, N=2, div=0, sigma=.2, is_put=False, n_paths = 1000)

Calculate the option price using the `.calcPrice()` method

In [17]:
optpricer.calcPrice()

5.153379851031563

In [20]:
optpricer = BSMOptionPricer(S0 = 50, K = 50, r=0.05, T=1, N=2, div=0, sigma=.2, is_put=True, n_paths = 1000)

In [21]:
optpricer.calcPrice()

2.613056176078671

View simulated stock price paths

In [10]:
optpricer.sim_paths

array([[50.        , 50.        , 50.        , ..., 50.        ,
        50.        , 50.        ],
       [47.51372062, 56.38168965, 40.47727001, ..., 34.05213183,
        57.73984185, 52.42540857],
       [58.47460184, 64.78876423, 33.78895213, ..., 37.6369312 ,
        55.43717076, 61.37393875]])